<img src="https://drive.google.com/uc?export=view&id=1wYSMgJtARFdvTt5g7E20mE4NmwUFUuog" width="200">

[![Gen AI Experiments](https://img.shields.io/badge/Gen%20AI%20Experiments-GenAI%20Bootcamp-blue?style=for-the-badge&logo=artificial-intelligence)](https://github.com/buildfastwithai/gen-ai-experiments)
[![Gen AI Experiments GitHub](https://img.shields.io/github/stars/buildfastwithai/gen-ai-experiments?style=for-the-badge&logo=github&color=gold)](http://github.com/buildfastwithai/gen-ai-experiments)


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1iHtEzN_Bb9nfN4TJL_H46sy7b_PoZO-w#scrollTo=lVkgCDANH0u_)

## Master Generative AI in 8 Weeks
**What You'll Learn:**
- Master cutting-edge AI tools & frameworks
- 6 weeks of hands-on, project-based learning
- Weekly live mentorship sessions
- No coding experience required
- Join Innovation Community
Transform your AI ideas into reality through hands-on projects and expert mentorship.
[Start Your Journey](https://www.buildfastwithai.com/genai-course)




# Top 3: Text [LMArena](https://lmarena.ai/leaderboard/text) Model Comparison

This notebook compares three top-performing models via OpenRouter:

- `google/gemini-2.5-pro`
- `openai/gpt-5`
- `anthropic/claude-opus-4.1`

We will:
- Do a quick primer on OpenRouter usage
- Set up a lightweight comparison harness
- Run demos: Q&A, summarization, information extraction, and coding help

Prerequisites:
- Get an OpenRouter API key and set environment variable `OPENROUTER_API_KEY`.
- Install `openai` (the new unified client) for simple API calls.

Note: Replace or parameterize models as availability/quotas may vary.


## Basic Setup

In [ ]:
#Installing Necessary libraries
!pip install --quiet openai python-dotenv --quiet

In [ ]:
# Setting Up Environment Varibles
from google.colab import userdata
OPENROUTER_API_KEY=userdata.get('OPENROUTER_API_KEY')

In [ ]:
import os
from typing import List, Dict, Any

from dotenv import load_dotenv
load_dotenv()

# Use the official OpenAI python client pointed to OpenRouter.
# Docs: https://openrouter.ai/docs
from openai import OpenAI

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY,
)

MODELS = [
    "google/gemini-2.5-pro",
    "openai/gpt-5",
    "anthropic/claude-opus-4.1",
]

SYSTEM_PRIMER = (
    "You are a helpful, concise assistant. Prefer clear bullet points and short code."
)

def call_model(model: str, user_prompt: str, system_prompt: str = SYSTEM_PRIMER, **kwargs: Any) -> str:
    """Call a single model with a chat completion and return its text output."""
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        **kwargs,
    )
    return response.choices[0].message.content

def compare_models(user_prompt: str, system_prompt: str = SYSTEM_PRIMER, **kwargs: Any) -> Dict[str, str]:
    """Run the same prompt across all models and return a mapping of model->output."""
    outputs: Dict[str, str] = {}
    for model in MODELS:
        try:
            outputs[model] = call_model(model, user_prompt, system_prompt, **kwargs)
        except Exception as e:
            outputs[model] = f"<error: {type(e).__name__}: {e}>"
    return outputs

def display_side_by_side(results: Dict[str, str]):
    """Nicely print results for quick visual comparison."""
    for model, text in results.items():
        print("\n" + "=" * 80)
        print(f"Model: {model}")
        print("-" * 80)
        print(text)
        print("\n")


## Quick OpenRouter usage primer

OpenRouter is an API gateway for many frontier models under one endpoint.

Basic call structure using the unified `openai` client:

```python
from openai import OpenAI
client = OpenAI(base_url="https://openrouter.ai/api/v1", api_key=OPENROUTER_API_KEY)
resp = client.chat.completions.create(
    model="openai/gpt-5",
    messages=[{"role":"user","content":"Say hi in one sentence"}]
)
print(resp.choices[0].message.content)
```

We’ll reuse this structure for all models by switching the `model` name.


In [ ]:
# Demo 1: General Q&A
prompt = "Explain the difference between concurrency and parallelism with a short example."
results = compare_models(prompt, temperature=0.2)
display_side_by_side(results)


In [ ]:
# Demo 2: Summarization
text = (
    """
OpenRouter provides access to many models like GPT, Claude, and Gemini via a single API.
It simplifies experimenting across providers and comparing outputs. With one client and
endpoint, you can switch `model` names to test speed, style, and quality differences.
This enables faster iteration and systematic evaluation.
"""
)

prompt = f"Summarize the following in 2-3 bullet points:\n\n{text}"
results = compare_models(prompt, temperature=0.2)
display_side_by_side(results)


In [ ]:
# Demo 3: Information extraction
snippet = "Order #A219 ships to 221B Baker Street, London on 2025-02-10. Total: $149.99 USD."

prompt = (
    "Extract a JSON object with keys: order_id, street, city, date, total_amount, currency. "
    f"Text: {snippet}"
)
results = compare_models(prompt, temperature=0)
display_side_by_side(results)


In [ ]:
# Demo 4: Coding assistance
prompt = (
    "Write a Python function `fizz_buzz(n)` that returns a list of strings from 1..n "
    "with 'Fizz' for multiples of 3, 'Buzz' for multiples of 5, and 'FizzBuzz' for multiples of both."
)
results = compare_models(prompt, temperature=0)
display_side_by_side(results)


### Notes and tips

- Ensure `OPENROUTER_API_KEY` is set. Using a `.env` file is supported via `python-dotenv`.
- Some models require provider-specific routing or may be rate-limited.
- For reproducibility, keep `temperature` low for deterministic tasks like extraction.
- Extend `MODELS` with other candidates from the Text Arena leaderboard.
- Consider adding simple cost/latency tracking if you need deeper evaluation.
